In [1]:
import rasterio, os

# import os
os.environ['USE_PYGEOS'] = '0'

import numpy as np
import pandas as pd 
import xarray as xr
import rioxarray as rx
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
from rasterio.transform import from_origin
from rasterio.mask import mask

In [13]:
my_lib = '/media/AsustorThree_Public/Tutorials/myClimPy/'
import sys
if my_lib not in sys.path:
    sys.path.append(my_lib)
# from mycdat.extreme import temporal_curve_24hr
from temporal_curve import temporal_curve_24hr

In [4]:
ARI = 100
ssps = [126, 245, 585]
pths = [5., 50., 95.]
years = [2030, 2040, 2050]

In [5]:
# read baseline
base_file = '/home/lusun/LuSUN/Data/riotinto_mash/all_durations/Australia_Extreme_Precipitation_baseline_DRDS_10080min.nc4'
ds = xr.open_dataset(base_file)
# variable_names = list(ds.variables.keys())
# variable_names

In [9]:
ds

<xarray.Dataset>
Dimensions:                (hrs: 1, ari: 11, lat: 1361, lon: 1761)
Coordinates:
  * lat                    (lat) float64 -9.787 -9.812 -9.837 ... -43.76 -43.79
  * lon                    (lon) float64 110.4 110.4 110.4 ... 154.3 154.4 154.4
  * hrs                    (hrs) float64 168.0
  * ari                    (ari) int64 2 3 5 10 20 50 100 200 500 1000 2000
Data variables:
    Extreme_Precipitation  (hrs, ari, lat, lon) float32 ...
Attributes:
    units:       mm
    regenerate:  http://www.climsystems.com J.Yu 2023-07-03
    source:      Australia Govemment Bureau of Meteorology Design Rainfall Da...

In [10]:
base_data = xr.open_dataset(base_file)['Extreme_Precipitation'].sel(ari=ARI).drop(["ari"]).rename({"lat": "latitude", "lon":"longitude"})
base_data.rio.write_crs("epsg:4326", inplace=True)
base_data

<xarray.DataArray 'Extreme_Precipitation' (hrs: 1, latitude: 1361,
                                           longitude: 1761)>
[2396721 values with dtype=float32]
Coordinates:
  * latitude     (latitude) float64 -9.787 -9.812 -9.837 ... -43.76 -43.79
  * longitude    (longitude) float64 110.4 110.4 110.4 ... 154.3 154.4 154.4
  * hrs          (hrs) float64 168.0
    spatial_ref  int64 0
Attributes:
    units:       mm
    regenerate:  http://www.climsystems.com J.Yu 2023-07-03
    source:      Australia Govemment Bureau of Meteorology Design Rainfall Da...

In [11]:
gdf_Pilbara = gpd.read_file('/home/lusun/LuSUN/Data/riotinto_mash/catchment_boundary/watershed_WGS84_without_offshore_islands_filledholes_dissolved.shp')
gdf_Pilbara

,id,geometry
0,1,"POLYGON ((118.74750 -20.29250, 118.74750 -20.3..."


In [18]:
# get regional DDF and PDF
pdfs_df = pd.DataFrame()
cdfs_df = pd.DataFrame()
ddfs_region = pd.DataFrame()

for i in gdf_Pilbara.index:
    
    gdf = gdf_Pilbara.iloc[[i]]
    # FID = i
    # region_name = gdf.iloc[0]['CZ']
    region_name = 'Pilbara'
    
    
    da_auk = base_data.rio.clip(gdf.geometry.values, gdf.crs)
    da_auk_m = da_auk.mean(dim=['latitude', "longitude"]).drop("spatial_ref")

    # to dataframe
    df_ddfs = da_auk_m.to_dataframe().rename({"Extreme_Precipitation" : "ddf"}, axis=1)


    # convert duration from hours to minutes
    df_ddfs.index = np.round(df_ddfs.index*60, 4)
    df_ddfs.index.name = "mins"
    
#     # Calculate temporal curve
#     df_crrs_auk_m = temporal_curve_24hr(df_ddfs)
    
#     # Convert CDF to PDF
#     crrs_cdf_series = df_crrs_auk_m['crr']
#     crrs_pdf_series = df_crrs_auk_m['crr'].diff().fillna(df_crrs_auk_m['crr'][0])
    
#     # cdf_series = df_crrs_auk_m * df_ddfs.loc[1440].values 
#     # pdf_series = cdf_series['crr'].diff().fillna(cdf_series['crr'][0])
#     # # pdf_df = pd.DataFrame({region_name: pdf_series})
    
#     print(region_name)
#     pdfs_df[region_name] = crrs_pdf_series
#     cdfs_df[region_name] = crrs_cdf_series
#     ddfs_region[region_name] = df_ddfs
    
    
# out_path = '/home/lusun/LuSUN/output/'
# if not os.path.exists(out_path):
#     os.makedirs(out_path)
    

                ddf
mins               
10080.0  323.042114
